### root에서 데이터베이스 사용하기.

In [ ]:
import pymysql

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect("localhost",
                     "root",
                     "root",
                     "news")

cursor = db.cursor()

sql = '''
   CREATE TABLE news(
       id INT UNSIGNED NOT NULL AUTO_INCREMENT,
       date VARCHAR(25) NOT NULL,
       company VARCHAR(15) NOT NULL,
       title VARCHAR(100) NOT NULL,
       content VARCHAR(10000) NOT NULL,
       PRIMARY KEY(id) 
);  
'''

cursor.execute(sql)
cursor.execute('desc news')
results = cursor.fetchall()

for result in results:
    print (result)

# db에 complete 하기.
db.commit()

# db 연결 닫기
db.close()

In [7]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pandas as pd
import pymysql

# 접속
db = pymysql.connect("localhost",
                     "root",
                     "root",
                     "news")

# 커서 가져오기
cursor = db.cursor()

def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    # 제목 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    news_detail.append(pdate)

    # news text
    _text = bsoup.select('#articleBodyContents')[
        0].get_text().replace('\n', " ")
    btext = _text.replace(
        "// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())

    # 신문사
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail

columns = ['날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2020.05.11"
e_date = "2020.05.15"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)

    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + \
        "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + \
        e_to + "%2Ca%3A&start=" + str(page)

    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    req = requests.get(url, headers=header)
    print(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    if soup.findAll("a", {"class": "_sp_each_url"}) == []:
        break

    for urls in soup.findAll("a", {"class": "_sp_each_url"}):
        try:
            if urls.attrs["href"].startswith("https://news.naver.com"):
                print(urls.attrs["href"])
                news_detail = get_news(urls.attrs["href"])
#      
                sql = """
                    INSERT INTO `news` (`date`, `company`, `title`, `content`) VALUES(%s, %s, %s, %s);
                """
#                 print(sql)
                # 날짜, 신문사, 제목, 내용
                cursor.execute(sql, (news_detail[1], news_detail[3], news_detail[0], news_detail[2]))
    
                db.commit()
        except Exception as e:
            print(e)
            continue
    page += 10
    
#     20번 돌리고 일단 stop
    if page == 211:
        break
db.close()

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.11&de=2020.05.15&nso=so%3Ar%2Cp%3Afrom20200511to20200515%2Ca%3A&start=1
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=005&aid=0001321594
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=005&aid=0001321594
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=002&aid=0002134275
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=002&aid=0002134275
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002269638
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002269638
11
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.11&de=2020.05.15&nso=so%3Ar%2Cp%3Afrom20200511to20200515%2Ca%3A&start=11
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=079&aid=0003360259
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=079&aid=000

(1406, "Data too long for column 'company' at row 1")
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003465400
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003465400
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009863804
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009863804
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004425112
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004425112
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009863802
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009863802
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=421&aid=0004639520
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=421&aid=0004639520
121
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639485
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639485
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639480
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639480
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004678976
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004678976
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=052&aid=0001440383
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=052&aid=0001440383
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639473
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0004639473
201
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.11&de=2020.05.15&nso=so%3Ar%2Cp%3Afrom20200511to202

### 1. pymysql connect() 메소드를 이용해서 db에 객체 전달하기.

In [2]:
import pymysql

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.165.127.62',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')


OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

### 2. pymysql CREATE TABLE news()

In [3]:
import pymysql

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.165.127.62',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# connection 객체로부터 cursr()메서드를 호출하여 Cursor 객체를 가져온다.
cursor = db.cursor()

# SQL 문 만들기
# int unsigned 는 id 같은 경우에는 음수를 사용하지 않으므로 그 범위가 양수로 옮겨간다. 굿
sql = '''
   CREATE TABLE news(
       id INT UNSIGNED NOT NULL AUTO_INCREMENT,
       date VARCHAR(25) NOT NULL,
       company VARCHAR(15) NOT NULL,
       title VARCHAR(100) NOT NULL,
       content VARCHAR(10000) NOT NULL,
       PRIMARY KEY(id) 
);  
'''

# 실행하기.
cursor.execute(sql)
cursor.execute('desc news')
results = cursor.fetchall()

for result in results:
    print (result)

# db에 complete 하기.
db.commit()

# db 연결 닫기
db.close()

('id', 'int(10) unsigned', 'NO', 'PRI', None, 'auto_increment')
('date', 'varchar(25)', 'NO', '', None, '')
('company', 'varchar(15)', 'NO', '', None, '')
('title', 'varchar(100)', 'NO', '', None, '')
('content', 'varchar(10000)', 'NO', '', None, '')


### 3. ec2 mysql 연동해서 data 를 insert하기 - placeholder 구문을 사용해서 특수처리해서 하기!

### startwidth method() : https://www.tutorialspoint.com/python/string_startswith.htm


In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pandas as pd
import pymysql

# 접속
db = pymysql.connect(host='54.165.127.62',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# 커서 가져오기
cursor = db.cursor()

def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    # 제목 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    news_detail.append(pdate)

    # news text
    _text = bsoup.select('#articleBodyContents')[
        0].get_text().replace('\n', " ")
    btext = _text.replace(
        "// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())

    # 신문사
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail

columns = ['날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2020.05.01"
e_date = "2020.05.08"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)

    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + \
        "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + \
        e_to + "%2Ca%3A&start=" + str(page)

    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    req = requests.get(url, headers=header)
    print(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    if soup.findAll("a", {"class": "_sp_each_url"}) == []:
        break

    for urls in soup.findAll("a", {"class": "_sp_each_url"}):
        try:
            if urls.attrs["href"].startswith("https://news.naver.com"):
                print(urls.attrs["href"])
                news_detail = get_news(urls.attrs["href"])
#      
                sql = """
                    INSERT INTO `news` (`date`, `company`, `title`, `content`) VALUES(%s, %s, %s, %s);
                """
#                 print(sql)
                # 날짜, 신문사, 제목, 내용
                cursor.execute(sql, (news_detail[1], news_detail[3], news_detail[0], news_detail[2]))
    
                db.commit()
        except Exception as e:
            print(e)
            continue
    page += 10
    
#     20번 돌리고 일단 stop
    if page == 211:
        break
db.close()

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.01&de=2020.05.08&nso=so%3Ar%2Cp%3Afrom20200501to20200508%2Ca%3A&start=1
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=469&aid=0000494602
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=469&aid=0000494602
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=311&aid=0001144127
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=311&aid=0001144127
list index out of range
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=056&aid=0010832394
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=056&aid=0010832394
11
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.01&de=2020.05.08&nso=so%3Ar%2Cp%3Afrom20200501to20200508%2Ca%3A&start=11
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=015&aid=0004337355
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&

https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.01&de=2020.05.08&nso=so%3Ar%2Cp%3Afrom20200501to20200508%2Ca%3A&start=81
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009853270
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=003&aid=0009853270
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002085860
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002085860
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=015&aid=0004337341
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=015&aid=0004337341
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=009&aid=0004571423
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=009&aid=0004571423
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=056&aid=0010832335
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=0

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597187
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597187
151
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2020.05.01&de=2020.05.08&nso=so%3Ar%2Cp%3Afrom20200501to20200508%2Ca%3A&start=151
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001036601
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001036601
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597184
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597184
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597186
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011597186
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=091&aid=0007998191
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&

###  4. pymysql data insert 한 것을 select 해서 dict, list에 저장하기.

In [6]:
import pymysql
import pandas as pd

columns = ['인덱스','날짜', '신문사', '제목', '내용']
# df = pd.DataFrame(columns=columns)
df = dict()
df_list = list()

# df = df.append(pd.DataFrame(
#     [[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]], columns=columns))

# pymysql.connect로 MYSQL 연결 객체를 db에 전달
db = pymysql.connect(host='54.165.127.62',
                     port=3306,
                     user='user',
                     passwd='goqudejr15',
                     db='mydb',
                     charset='utf8')

# connection 객체로부터 cursr()메서드를 호출하여 Cursor 객체를 가져온다.
cursor = db.cursor()

sql = """
    SELECT * FROM news
"""

# 실행하기.
# cursor.execute(sql)
cursor.execute(sql)
results = cursor.fetchall()

for index, result in enumerate(results):
    
    index = result[0]
    date = result[1]
    company = result[2]
    title = result[3]
    content = result[4]

    df = {
        '인덱스' : index,
        '날짜' : date,
        '신문사' : company,
        '제목' : title,
        '내용' : content}
          
    df_list.append(df)

# for result in results:
#     print (result[0], result[1], result[2], result[3], result[4])
#     df = df.append(pd.DataFrame([[result[0], result[1], result[2], result[3], result[4]]], columns=columns))
    
    
# db에 complete 하기.
db.commit()

# db 연결 닫기
db.close()
print(df_list)

[{'인덱스': 1, '날짜': '2020.05.08', '신문사': '한국일보', '제목': '[속보] 청주에서도 이태원 클럽 관련 신종 코로나 환자', '내용': '7일 오후 신종코로나 바이러스 감염증(코로나19) 확진자가 다녀간 서울 이태원의 한 클럽 앞에서 기자들이 취재를 하고 있다. 연합뉴스서울 이태원 클럽을 방문한 사람 가운데서 신종 코로나바이러스 감염증(코로나19) 확진 환자가 증가하고 있다. 청주에서 거주하는 20대 남성까지 확진판정을 받았다. 8일 충북도 등에 따르면 청주에 사는 A(22)씨가 이날 코로나19 확진 판정을 받았다. A씨는 지난 4일 서울 용산구 이태원의 클럽을 다녀간 것으로 전해졌다. A씨는 이태원 클럽에 함께 다녀온 친구가 확진 판정을 받자 8일 보건소를 찾아 진단 검사를 받은 것으로 알려졌다. A씨는 청주에서 발생한 14번째 환자다. 충북의 코로나19 환자는 A씨를 포함해 현재까지 48명이다.김민호 기자 kmh@hankookilbo.com\xa0▶네이버에서 한국일보 뉴스를 받아보세요\xa0▶뉴;잼을 보면 뉴스가 재밌다 [뉴스레터 구독하기]\xa0▶한국일보닷컴 바로가기'}, {'인덱스': 2, '날짜': '2020.05.08', '신문사': 'KBS', '제목': '중국, ‘코로나19 일상복귀’ 영화관 등 재개장 결정', '내용': "중국 정부가 자국 내의 코로나19 확산세가 통제 가능한 범위에 들어왔다는 판단에 따라 영화관 등 다중 이용 시설을 재개장할 수 있도록 결정했습니다. 인민일보 등 중국매체의 오늘(9일 어제) 보도에 따르면 중국 국무원의 코로나19 합동 예방통제체제는 최근 '코로나19 정상화 방역작업 지도의견'을 통해 이같은 내용을 발표했습니다. 이 발표에 따르면 방역 조치를 한다는 전제하에 백화점과 슈퍼마켓, 호텔과 식당 등 생활시설은 전면 개방할 수 있도록 했으며, 예약제 등의 방식에 따라 공원과 관광명소, 운동시설과 도서관, 그리고 박물관과 미술관 등  실내시설과 영화관과 오락실 등 밀폐된 오락 여가 시설도 개

In [8]:
testlist = []

for news in df_list:
    print (news['내용'])

7일 오후 신종코로나 바이러스 감염증(코로나19) 확진자가 다녀간 서울 이태원의 한 클럽 앞에서 기자들이 취재를 하고 있다. 연합뉴스서울 이태원 클럽을 방문한 사람 가운데서 신종 코로나바이러스 감염증(코로나19) 확진 환자가 증가하고 있다. 청주에서 거주하는 20대 남성까지 확진판정을 받았다. 8일 충북도 등에 따르면 청주에 사는 A(22)씨가 이날 코로나19 확진 판정을 받았다. A씨는 지난 4일 서울 용산구 이태원의 클럽을 다녀간 것으로 전해졌다. A씨는 이태원 클럽에 함께 다녀온 친구가 확진 판정을 받자 8일 보건소를 찾아 진단 검사를 받은 것으로 알려졌다. A씨는 청주에서 발생한 14번째 환자다. 충북의 코로나19 환자는 A씨를 포함해 현재까지 48명이다.김민호 기자 kmh@hankookilbo.com ▶네이버에서 한국일보 뉴스를 받아보세요 ▶뉴;잼을 보면 뉴스가 재밌다 [뉴스레터 구독하기] ▶한국일보닷컴 바로가기
중국 정부가 자국 내의 코로나19 확산세가 통제 가능한 범위에 들어왔다는 판단에 따라 영화관 등 다중 이용 시설을 재개장할 수 있도록 결정했습니다. 인민일보 등 중국매체의 오늘(9일 어제) 보도에 따르면 중국 국무원의 코로나19 합동 예방통제체제는 최근 '코로나19 정상화 방역작업 지도의견'을 통해 이같은 내용을 발표했습니다. 이 발표에 따르면 방역 조치를 한다는 전제하에 백화점과 슈퍼마켓, 호텔과 식당 등 생활시설은 전면 개방할 수 있도록 했으며, 예약제 등의 방식에 따라 공원과 관광명소, 운동시설과 도서관, 그리고 박물관과 미술관 등  실내시설과 영화관과 오락실 등 밀폐된 오락 여가 시설도 개방할 수 있도록 했습니다.  한편 중국 국무원은 사람들이 밀집한 밀폐공간에서는 타인과 1미터 이내에 있을 경우 마스크를 쓰도록 하는 등의 권고 사항도 발표했습니다.[사진 출처 : 로이터=연합뉴스]▶ ‘ 코로나19 확산 우려’ 최신 기사 보기http://news.kbs.co.kr/news/list.do?icd=19588김동진 기자 (grandkdj@gm